# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets_1 import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 81545040, 'calculationPrice': 'close', 'change': 3.3, 'changePercent': 0.02475, 'close': 0, 'closeSource': 'iffcloai', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 141.3, 'iexCloseTime': 1671703141236, 'iexLastUpdated': 0, 'iexMarketPercent': None, 'iexOpen': None, 'iexOpenTime': None, 'iexRealtimePrice': 0, 'iexRealtimeSize': 0, 'iexVolume': 0, 'lastTradeTime': 1717396852893, 'latestPrice': 137.36, 'latestSource': 'Close', 'latestTime': 'December 21, 2022', 'latestUpdate': 1718447194548, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2193826491493, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'open': 0, 'openTime': Non

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [8]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

22.19

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [9]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [13]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["quote"]["latestPrice"],
                    data[symbol]["stats"]["year1ChangePercent"],
                    "N/A",
                ],
                index=my_columns,
            ),
            ignore_index=True,
        )


final_dataframe


/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_22543/2771355120.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_22543/2771355120.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_22543/2771355120.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_22543/2771355120.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,150.53,-0.019739,N/A
1,AAL,13.38,-0.289603,N/A
2,AAP,143.59,-0.363117,N/A
3,AAPL,142.09,-0.216990,N/A
4,ABBV,163.49,0.298144,N/A
...,...,...,...,...
95,CINF,108.28,-0.041269,N/A
96,CL,81.07,-0.014532,N/A
97,CLX,147.80,-0.138241,N/A
98,CMA,65.53,-0.210657,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [17]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:10]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,level_0,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,0,APA,48.60,0.893093,N/A
1,1,CAH,83.00,0.698903,N/A
2,2,CI,346.86,0.547471,N/A
3,3,ADM,98.75,0.494412,N/A
4,4,CF,93.91,0.402265,N/A
5,5,ABC,171.46,0.359074,N/A
6,6,ABBV,163.49,0.298144,N/A
7,7,AFL,72.70,0.292179,N/A
8,8,ANTM,502.52,0.292029,N/A
9,9,BKR,29.44,0.272530,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [18]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [19]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [21]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
final_dataframe

,level_0,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,0,APA,48.60,0.893093,2057
1,1,CAH,83.00,0.698903,1204
2,2,CI,346.86,0.547471,288
3,3,ADM,98.75,0.494412,1012
4,4,CF,93.91,0.402265,1064
5,5,ABC,171.46,0.359074,583
6,6,ABBV,163.49,0.298144,611
7,7,AFL,72.70,0.292179,1375
8,8,ANTM,502.52,0.292029,198
9,9,BKR,29.44,0.272530,3396


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [28]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


17.231212133061742

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [30]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

rv_dataframe

/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_22543/1288680314.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_22543/1288680314.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_22543/1288680314.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_22543/1288680314.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_da

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,150.35,N/A,35.8,N/A,8.37,N/A,6.51,N/A,23.621808,N/A,12.672382,N/A,N/A
1,AAL,12.98,N/A,-5.34,N/A,-1.08,N/A,0.1827,N/A,19.399080,N/A,0.734845,N/A,N/A
2,AAP,148.96,N/A,17.14,N/A,3.1,N/A,0.784,N/A,9.555078,N/A,2.010091,N/A,N/A
3,AAPL,134.59,N/A,21.71,N/A,41.9,N/A,5.35,N/A,16.603657,N/A,12.995972,N/A,N/A
4,ABBV,170.90,N/A,22.08,N/A,18.67,N/A,5.01,N/A,12.540232,N/A,8.770669,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,107.29,N/A,-589.97,N/A,1.8,N/A,2.47,N/A,-565.593644,N/A,2.365673,N/A,N/A
96,CL,82.06,N/A,35.33,N/A,107.26,N/A,3.9,N/A,19.582691,N/A,7.206897,N/A,N/A
97,CLX,147.30,N/A,44.57,N/A,56.46,N/A,2.54,N/A,29.044387,N/A,8.014327,N/A,N/A
98,CMA,67.81,N/A,8.79,N/A,1.93,N/A,2.65,N/A,6.294534,N/A,2.554745,N/A,N/A


{'week52change': 0.290414, 'week52high': 169.51, 'week52low': 90.41, 'marketcap': 75861250162, 'employees': 10724, 'day200MovingAvg': 134.17, 'day50MovingAvg': 148.64, 'float': 479955020, 'avg10Volume': 1641586.3, 'avg30Volume': 1479579.2, 'ttmEPS': 3.499, 'ttmDividendRate': 0.77, 'companyName': 'Zoetis, Inc.', 'sharesOutstanding': 494883276, 'maxChangePercent': 3.3525, 'year5ChangePercent': 1.69, 'year2ChangePercent': 0.608, 'year1ChangePercent': 0.281583, 'ytdChangePercent': 0.189876, 'month6ChangePercent': 0.10241, 'month3ChangePercent': 0.297048, 'month1ChangePercent': 0.159224, 'day30ChangePercent': 0.165634, 'day5ChangePercent': -0.012817, 'nextDividendDate': None, 'dividendYield': 0.00489706569361443, 'nextEarningsDate': '2020-10-13', 'exDividendDate': '2020-07-01', 'peRatio': 49.1, 'beta': 0.9584403132945124, 'totalCash': 1974321215, 'currentDebt': 549171552, 'revenue': 6518431250, 'grossProfit': 4133660573, 'totalRevenue': 6300683466, 'EBITDA': 2502772553, 'revenuePerShare': 1

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [33]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
6,ABMD,384.12,N/A,66.57,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
28,ALXN,189.70,N/A,61.77,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,307.00,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,310.72,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,96.58,N/A,48.66,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [41]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,150.35,N/A,35.80,N/A,8.37,N/A,6.5100,N/A,23.621808,N/A,12.672382,N/A,N/A
1,AAL,12.98,N/A,-5.34,N/A,-1.08,N/A,0.1827,N/A,19.399080,N/A,0.734845,N/A,N/A
2,AAP,148.96,N/A,17.14,N/A,3.10,N/A,0.7840,N/A,9.555078,N/A,2.010091,N/A,N/A
3,AAPL,134.59,N/A,21.71,N/A,41.90,N/A,5.3500,N/A,16.603657,N/A,12.995972,N/A,N/A
4,ABBV,170.90,N/A,22.08,N/A,18.67,N/A,5.0100,N/A,12.540232,N/A,8.770669,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,107.29,N/A,-589.97,N/A,1.80,N/A,2.4700,N/A,-565.593644,N/A,2.365673,N/A,N/A
96,CL,82.06,N/A,35.33,N/A,107.26,N/A,3.9000,N/A,19.582691,N/A,7.206897,N/A,N/A
97,CLX,147.30,N/A,44.57,N/A,56.46,N/A,2.5400,N/A,29.044387,N/A,8.014327,N/A,N/A
98,CMA,67.81,N/A,8.79,N/A,1.93,N/A,2.6500,N/A,6.294534,N/A,2.554745,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [43]:
rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [44]:
from scipy.stats import percentileofscore as score

metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,150.35,N/A,35.80,0.81,8.37,0.81,6.5100,0.87,23.621808,0.88,12.672382,0.91,N/A
1,AAL,12.98,N/A,-5.34,0.08,-1.08,0.05,0.1827,0.03,19.399080,0.79,0.734845,0.05,N/A
2,AAP,148.96,N/A,17.14,0.39,3.10,0.4,0.7840,0.12,9.555078,0.32,2.010091,0.15,N/A
3,AAPL,134.59,N/A,21.71,0.54,41.90,0.97,5.3500,0.78,16.603657,0.66,12.995972,0.92,N/A
4,ABBV,170.90,N/A,22.08,0.56,18.67,0.92,5.0100,0.77,12.540232,0.45,8.770669,0.68,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,107.29,N/A,-589.97,0.01,1.80,0.2,2.4700,0.44,-565.593644,0.01,2.365673,0.2,N/A
96,CL,82.06,N/A,35.33,0.8,107.26,1.0,3.9000,0.66,19.582691,0.8,7.206897,0.53,N/A
97,CLX,147.30,N/A,44.57,0.88,56.46,0.99,2.5400,0.45,29.044387,0.97,8.014327,0.61,N/A
98,CMA,67.81,N/A,8.79,0.18,1.93,0.245,2.6500,0.47,6.294534,0.14,2.554745,0.22,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [46]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,150.35,N/A,35.80,0.81,8.37,0.81,6.5100,0.87,23.621808,0.88,12.672382,0.91,0.856
1,AAL,12.98,N/A,-5.34,0.08,-1.08,0.05,0.1827,0.03,19.399080,0.79,0.734845,0.05,0.2
2,AAP,148.96,N/A,17.14,0.39,3.10,0.4,0.7840,0.12,9.555078,0.32,2.010091,0.15,0.276
3,AAPL,134.59,N/A,21.71,0.54,41.90,0.97,5.3500,0.78,16.603657,0.66,12.995972,0.92,0.774
4,ABBV,170.90,N/A,22.08,0.56,18.67,0.92,5.0100,0.77,12.540232,0.45,8.770669,0.68,0.676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,107.29,N/A,-589.97,0.01,1.80,0.2,2.4700,0.44,-565.593644,0.01,2.365673,0.2,0.172
96,CL,82.06,N/A,35.33,0.8,107.26,1.0,3.9000,0.66,19.582691,0.8,7.206897,0.53,0.758
97,CLX,147.30,N/A,44.57,0.88,56.46,0.99,2.5400,0.45,29.044387,0.97,8.014327,0.61,0.78
98,CMA,67.81,N/A,8.79,0.18,1.93,0.245,2.6500,0.47,6.294534,0.14,2.554745,0.22,0.251


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [50]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,65.18,N/A,3.870000,0.11,1.240000,0.12,0.809600,0.14,2.204540,0.04,0.791363,0.06,0.094
1,BA,193.66,N/A,-13.380000,0.06,-6.620000,0.04,1.870000,0.35,-29.500290,0.02,-185.135307,0.01,0.096
2,C,44.36,N/A,6.250000,0.15,0.480800,0.06,0.992900,0.17,3.521327,0.07,0.956910,0.08,0.106
3,ALL,139.97,N/A,-119.710000,0.02,2.350000,0.345,0.742700,0.11,5.469113,0.1,0.702395,0.04,0.123
4,CAH,83.79,N/A,-20.180000,0.05,-12.270000,0.02,0.112500,0.01,9.067029,0.3,3.522399,0.27,0.13
5,CCL,7.83,N/A,-1.260000,0.1,1.070000,0.09,0.925300,0.16,-13.678449,0.03,3.611433,0.28,0.132
6,BEN,27.14,N/A,8.310000,0.17,1.170000,0.11,1.580000,0.27,5.598528,0.11,1.440931,0.1,0.152
7,CINF,107.29,N/A,-589.970000,0.01,1.800000,0.2,2.470000,0.44,-565.593644,0.01,2.365673,0.2,0.172
8,AIZ,125.12,N/A,21.360000,0.51,1.680000,0.18,0.675100,0.1,4.940958,0.09,0.633675,0.03,0.182
9,AAL,12.98,N/A,-5.340000,0.08,-1.080000,0.05,0.182700,0.03,19.399080,0.79,0.734845,0.05,0.2


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [51]:
portfolio_input()

In [55]:
position_size = float(portfolio_size)/len(rv_dataframe.index)
for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AIG,65.18,7671,3.870000,0.11,1.240000,0.12,0.809600,0.14,2.204540,0.04,0.791363,0.06,0.094
1,BA,193.66,2581,-13.380000,0.06,-6.620000,0.04,1.870000,0.35,-29.500290,0.02,-185.135307,0.01,0.096
2,C,44.36,11271,6.250000,0.15,0.480800,0.06,0.992900,0.17,3.521327,0.07,0.956910,0.08,0.106
3,ALL,139.97,3572,-119.710000,0.02,2.350000,0.345,0.742700,0.11,5.469113,0.1,0.702395,0.04,0.123
4,CAH,83.79,5967,-20.180000,0.05,-12.270000,0.02,0.112500,0.01,9.067029,0.3,3.522399,0.27,0.13
5,CCL,7.83,63856,-1.260000,0.1,1.070000,0.09,0.925300,0.16,-13.678449,0.03,3.611433,0.28,0.132
6,BEN,27.14,18422,8.310000,0.17,1.170000,0.11,1.580000,0.27,5.598528,0.11,1.440931,0.1,0.152
7,CINF,107.29,4660,-589.970000,0.01,1.800000,0.2,2.470000,0.44,-565.593644,0.01,2.365673,0.2,0.172
8,AIZ,125.12,3996,21.360000,0.51,1.680000,0.18,0.675100,0.1,4.940958,0.09,0.633675,0.03,0.182
9,AAL,12.98,38520,-5.340000,0.08,-1.080000,0.05,0.182700,0.03,19.399080,0.79,0.734845,0.05,0.2


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [59]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)


## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [60]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [61]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [62]:
writer.save()

/var/folders/gt/v_9gvkrj0cb8yzv54yz0fg8m0000gn/T/ipykernel_22543/934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
